<a href="https://colab.research.google.com/github/ardabyr/DataScience/blob/main/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_%D1%83%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BF%D1%80%D0%BE%D0%BF%D1%83%D1%81%D0%BA%D0%BE%D0%B2_%D0%B8_%D0%BF%D1%80%D0%B5%D0%BE%D0%B4%D0%BE%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D0%B0%D0%BD%D0%BE%D0%BC%D0%B0%D0%BB%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Вариант №4.**
ecoli.csv

Проведите анализ пропусков и аномалий для набора данных, заданного по варианту. Все датасеты взяты из репозитория [KEEL](https://sci2s.ugr.es/keel/datasets.php)
Обязательно при выполнении задания необходимо:


*   Загрузить данные. Предварительно необходимо оставить только числовые признаки, если есть закодированные текстом категориальные признаки, то применить конвертацию категориальных признаков в числовые (one hot enconding или label encoding).
*   Применить метод для внесения пропусков в загруженный набор данных. Целевая переменная не должна использоваться при использовании этого метода.
*   Сформировать два набора признаков: если пропуски в количественном признаке, то с внесением фиксированного значения вместо пропусков и с внесением среднего значения вместо пропусков. Если пропуски появились в категориальном признаке, то с внесением фиксированного значения вместо пропусков и с внесением самого часто встречающегося значения вместо пропусков.
*   Обучить классификатор Логистическая регрессия на имеющихся признаках с применением кросс-валидации. Оценить точность классификации для разных методов устранения пропусков на тестовом наборе данных.
*   Определите, сколько данных (% от общего количества) можно отнести к выбросам, если задать коэффициент k (умножаемый на межквартильное расстояние) = 1.9? Выбросы определить для каждого признака по отдельности, а затем результаты объединить. Целевая переменная не анализируется на выбросы. Используйте набор данных без пропущенных значений.
*   Удалите из набора данные, которые отнесены к выбросам и оцените полученную точность классификации на тестовом наборе (необходимо использовать набор без внесения пропусков данных). Нужно учесть, что выбросы определяются и удаляются только из train части набора данных.



In [ ]:
!git lfs clone https://github.com/Mereep/keel_csv.git

In [ ]:
import pandas as pd
df = pd.read_csv('keel_csv/standard_classification/unprocessed/ecoli.csv', sep=',')

Загрузить данные. Предварительно необходимо оставить только числовые признаки, если есть закодированные текстом категориальные признаки, то применить конвертацию категориальных признаков в числовые (one hot enconding или label encoding).


In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Создать объект label encoder
le = LabelEncoder()

df["Site"] = le.fit_transform(df["Site"])

In [ ]:
df.head()

,Mcg,Gvh,Lip,Chg,Aac,Alm1,Alm2,Site
0,49.0,29.0,48.0,5.0,56.0,24.0,35.0,0
1,7.0,4.0,48.0,5.0,54.0,35.0,44.0,0
2,56.0,4.0,48.0,5.0,49.0,37.0,46.0,0
3,59.0,49.0,48.0,5.0,52.0,45.0,36.0,0
4,23.0,32.0,48.0,5.0,55.0,25.0,35.0,0


In [ ]:
df.isnull().sum()

Mcg     0
Gvh     0
Lip     0
Chg     0
Aac     0
Alm1    0
Alm2    0
Site    0
dtype: int64

Применить метод для внесения пропусков в загруженный набор данных. Целевая переменная не должна использоваться при использовании этого метода.


In [ ]:
variant = 4 # Сюда вставляем номер варианта

def createNA(X):

  np.random.seed(variant)
  del_num = np.arange(X.shape[1])
  np.random.shuffle(del_num)
  for i in range(2):
    nan_num = np.arange(X.shape[0])
    np.random.shuffle(nan_num)
    nan_count = round((X.shape[0]*0.05)) + 1
    nan_num = nan_num[0:nan_count]
    for index,row in X.iterrows():
      if index in nan_num:
          X.iat[index,del_num[i]]=None
  return X

X = df.drop("Site", axis=1)
createNA(X)
y = df['Site']

Сформировать два набора признаков: если пропуски в количественном признаке, то с внесением фиксированного значения вместо пропусков и с внесением среднего значения вместо пропусков. Если пропуски появились в категориальном признаке, то с внесением фиксированного значения вместо пропусков и с внесением самого часто встречающегося значения вместо пропусков.


In [ ]:
X.isnull().sum()

Mcg      0
Gvh      0
Lip      0
Chg      0
Aac     18
Alm1     0
Alm2    18
dtype: int64

Aac, Alm2 - количественные признаки

In [ ]:
# Внести фиксированное значение вместо пропусков в количественные признаки
df_imp_constant = X.fillna(0)

# Внести среднего значения вместо пропусков в количественные признаки
df_imp_mean = X.fillna(X.mean())

In [ ]:
df_imp_mean

,Mcg,Gvh,Lip,Chg,Aac,Alm1,Alm2
0,49.0,29.0,48.0,5.0,56.0,24.0,35.000000
1,7.0,4.0,48.0,5.0,54.0,35.0,44.000000
2,56.0,4.0,48.0,5.0,49.0,37.0,46.000000
3,59.0,49.0,48.0,5.0,52.0,45.0,36.000000
4,23.0,32.0,48.0,5.0,55.0,25.0,35.000000
...,...,...,...,...,...,...,...
331,74.0,56.0,48.0,5.0,47.0,68.0,3.000000
332,71.0,57.0,48.0,5.0,48.0,35.0,32.000000
333,61.0,6.0,48.0,5.0,44.0,39.0,38.000000
334,59.0,61.0,48.0,5.0,42.0,42.0,37.000000


Обучить классификатор Логистическая регрессия на имеющихся признаках с применением кросс-валидации. Оценить точность классификации для разных методов устранения пропусков на тестовом наборе данных.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

#Разделим на обучающую и тестовую выборку
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop("Site", axis=1), y, test_size=0.8, random_state=42)

X_train_constant, X_test_constant, y_train_constant, y_test_constant = train_test_split(df_imp_constant, y, test_size=0.8, random_state=42)

X_train_mean, X_test_mean, y_train_mean, y_test_mean = train_test_split(df_imp_mean, y, test_size=0.8, random_state=42)

# оубчим модель на исходном наборе данных
model = LogisticRegression()
scores_original = cross_val_score(model, X_train, y_train, cv=5)
score_original = model.fit(X_train, y_train)

# обучим модель на наборе данных с внесением фиксированных значений
model = LogisticRegression()
scores_imp_constant = cross_val_score(model, X_train_constant, y_train_constant, cv=5)
score_original_constant = model.fit(X_train_constant, y_train_constant)

# обучим модель на наборе данных с внесением средних значений
model = LogisticRegression()
scores_imp_mean = cross_val_score(model, X_train_mean, y_train_mean, cv=5)
score_original_mean = model.fit(X_train_mean, y_train_mean)

In [ ]:
from sklearn.metrics import accuracy_score

print('Точность классификации на тестовых данных для исходного набора данных:', score_original.score(X_test, y_test))
print('Точность классификации на тестовых данных для набора данных с внесением фиксированных значений:', score_original_constant.score(X_test_constant, y_test_constant))
print('Точность классификации на тестовых данных для набора данных с внесением усредненных значений:', score_original_mean.score(X_test_mean, y_test_mean))

Точность классификации на тестовых данных для исходного набора данных: 0.6505576208178439
Точность классификации на тестовых данных для набора данных с внесением фиксированных значений: 0.7137546468401487
Точность классификации на тестовых данных для набора данных с внесением усредненных значений: 0.6579925650557621



Определите, сколько данных (% от общего количества) можно отнести к выбросам, если задать коэффициент k (умножаемый на межквартильное расстояние) = 1.9? Выбросы определить для каждого признака по отдельности, а затем результаты объединить. Целевая переменная не анализируется на выбросы. Используйте набор данных без пропущенных значений.

In [ ]:
for feature in X:

    iqr = df.select_dtypes(include=[np.number]).apply(lambda x: x.quantile(0.75) - x.quantile(0.25))

    outliers = ((df.select_dtypes(include=[np.number]) > (df.select_dtypes(include=[np.number]).mean() + 1.9 * iqr)) |
                (df.select_dtypes(include=[np.number]) < (df.select_dtypes(include=[np.number]).mean() - 1.9 * iqr))).any(axis=1)

    print(f"Признак: {feature}")
    print(f"Межквартильное расстояние: {iqr}")
    print(f"Выбросы: {outliers}n")


Признак: Mcg
Межквартильное расстояние: Mcg     36.00
Gvh     19.00
Lip      0.00
Chg      0.00
Aac     16.00
Alm1    38.25
Alm2    36.00
Site     4.00
dtype: float64
Выбросы: 0      True
1      True
2      True
3      True
4      True
       ... 
331    True
332    True
333    True
334    True
335    True
Length: 336, dtype: booln
Признак: Gvh
Межквартильное расстояние: Mcg     36.00
Gvh     19.00
Lip      0.00
Chg      0.00
Aac     16.00
Alm1    38.25
Alm2    36.00
Site     4.00
dtype: float64
Выбросы: 0      True
1      True
2      True
3      True
4      True
       ... 
331    True
332    True
333    True
334    True
335    True
Length: 336, dtype: booln
Признак: Lip
Межквартильное расстояние: Mcg     36.00
Gvh     19.00
Lip      0.00
Chg      0.00
Aac     16.00
Alm1    38.25
Alm2    36.00
Site     4.00
dtype: float64
Выбросы: 0      True
1      True
2      True
3      True
4      True
       ... 
331    True
332    True
333    True
334    True
335    True
Length: 336, dtype: bool


Удалите из набора данные, которые отнесены к выбросам и оцените полученную точность классификации на тестовом наборе (необходимо использовать набор без внесения пропусков данных). Нужно учесть, что выбросы определяются и удаляются только из train части набора данных.

In [ ]:
data = pd.DataFrame()
for feature in df:
    iqr = X_train.select_dtypes(include=[np.number]).apply(lambda x: x.quantile(0.75) - x.quantile(0.25))
    outliers = ((X_train.select_dtypes(include=[np.number]) > (X_train.select_dtypes(include=[np.number]).mean() + 1.9 * iqr)) |
                (X_train.select_dtypes(include=[np.number]) < (X_train.select_dtypes(include=[np.number]).mean() - 1.9 * iqr))).any(axis=1)
    X_train = X_train[outliers]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# оубчим модель на исходном наборе данных
model = LogisticRegression()
scores_original = cross_val_score(model, X_train, y_train, cv=5)
score = model.fit(X_test, y_test).score(X_test, y_test)

In [ ]:
score

0.8141263940520446

Точность классификации с удаленными выбросами: 0.8141263940520446